In [1]:
import json
import pprint
import csv
import numpy as np

In [4]:
columns = ['Id', 'UserId', 'Stars', 'Date', 'BusinessId'] #create required columns 
question_dataframe_file = open('./output/reviews.csv', 'wb')
writer = csv.writer(question_dataframe_file, delimiter=',')
writer.writerow(columns)
c = 0
with open('./yelp_dataset/yelp_academic_dataset_review.json') as data_file:
    for line in data_file:
        c+=1
        review = json.loads(line)
        writer.writerow([review['review_id'], review['user_id'], review['stars'], review['date'], review['business_id'] ] )
        if c%100000==0:
            print c
print c


KeyboardInterrupt: 

In [24]:
########################################
# flatten dict keys in bussiness.csv
########################################
c = 0
cols = []
with open('./yelp_dataset/yelp_academic_dataset_business.json') as data_file:
    for line in data_file:           
        c+=1
        review = json.loads(line)
        #print review
        for catg in review:
            if type(review[catg]) is dict:
                for item in review[catg]:
                    if type(review[catg][item]) is dict:
                        for nesteditem in review[catg][item]:
                            toappend = catg+":"+item+":"+nesteditem
                            if toappend not in cols:
                                cols.append(toappend)
                                #print toappend
                    else:
                        toappend = catg+":"+item
                        if toappend not in cols:
                            cols.append(toappend)
                            #print toappend
            elif catg not in cols:
                cols.append(catg)
                #print catg
#         pprint.pprint(review)
#         if c%100000==0:
        #print c
#         break
print c
print len(cols)



77445
98


In [27]:
cols.sort()

business_dataframe = open('./output/business.csv', 'wb')
writer = csv.writer(business_dataframe, delimiter=',')
writer.writerow(cols)

business_dataframe.close()

In [ ]:
with open('./yelp_dataset/yelp_academic_dataset_business.json') as data_file:
    for line in data_file:           
        c+=1
        review = json.loads(line)
        # convert each object's attributes to row values
        # writer.writerow()
        break
print c

In [ ]:
########################################
# flatten dict reference
########################################
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            #recursive call
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

'''
e.g.
>>> flatten({'a': 1, 'c': {'a': 2, 'b': {'x': 5, 'y' : 10}}, 'd': [1, 2, 3]})
{'a': 1, 'c_a': 2, 'c_b_x': 5, 'd': [1, 2, 3], 'c_b_y': 10}
'''

In [3]:
print cols

[u'city', u'review_count', u'name', u'neighborhoods', u'type', u'business_id', u'full_address', u'hours:Tuesday:close', u'hours:Tuesday:open', u'hours:Friday:close', u'hours:Friday:open', u'hours:Monday:close', u'hours:Monday:open', u'hours:Wednesday:close', u'hours:Wednesday:open', u'hours:Thursday:close', u'hours:Thursday:open', u'state', u'longitude', u'stars', u'latitude', u'attributes:Take-out', u'attributes:Drive-Thru', u'attributes:Outdoor Seating', u'attributes:Caters', u'attributes:Noise Level', u'attributes:Parking:garage', u'attributes:Parking:street', u'attributes:Parking:validated', u'attributes:Parking:lot', u'attributes:Parking:valet', u'attributes:Delivery', u'attributes:Price Range', u'attributes:Attire', u'attributes:Has TV', u'attributes:Good For:dessert', u'attributes:Good For:latenight', u'attributes:Good For:lunch', u'attributes:Good For:dinner', u'attributes:Good For:breakfast', u'attributes:Good For:brunch', u'attributes:Takes Reservations', u'attributes:Ambienc

In [2]:
columns = ['BusinessId', '0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7-8', '8-9', '9-10',
          '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19','19-20', '20-21','21-22',
          '22-23', '23-0'] #create required columns 
question_dataframe_file = open('./output/checkin2.csv', 'wb')
writer = csv.writer(question_dataframe_file, delimiter=',')
writer.writerow(columns)
c = 0
with open('./yelp_dataset/yelp_academic_dataset_checkin.json') as data_file:
    for line in data_file:
        times = np.zeros(24)
        c+=1
        review = json.loads(line)
        keys = review['checkin_info'].keys()
        #print keys
        for key in keys:
            #print hours
            #print key
            #print review['checkin_info'][key]
            hours = int(key.split('-')[0])
            times[hours] += review['checkin_info'][key]
            
         
        writer.writerow([review['business_id'], times[0], times[1], times[2], times[3], times[4], times[5],
                         times[6], times[7], times[8], times[9], times[10], times[11], times[12], times[13], times[14],
                          times[15], times[16],  times[17], times[18], times[19], times[20], times[21], times[22], times[23]])

print c

55569


In [12]:
import matplotlib.pyplot as plt
import pandas
#from pandas import DataFrame
%matplotlib inline

df = pandas.read_csv('./output/checkin2.csv')
#print df
df2 = df.BusinessId
print df2
# print df
# plt.plot(df[,not 'BussinessId'])
# plt.xlabel('Date')
# plt.xlim('Jan 2015' , 'Dec 2015')
# plt.ylabel('Posts')

0        cE27W9VPgO88Qxe4ol6y_g
1        mVHrayjG3uZ_RLHkLj-AMg
2        KayYbHCt-RkbGcPdGOThNg
3        b12U9TFESStdy7CsTtcOeg
4        QoDa50dc7g62xciFygXB9w
5        1_lU0-eSWJCRvNGk78Zh9Q
6        _qopVQ6_Mz6W7-Pmbi56GQ
7        wJr6kSA5dchdgOdwH6dZ2w
8        yXuao0pFz1AxB21vJjDf5w
9        fNGIbpazjTRdXgwRY_NIXA
10       b9WZJp5L1RZr4F1nxclOoQ
11       zaXDakTd3RXyOa7sMrUE1g
12       zgy27FSnvwdINfk5cXBIyQ
13       WETE_LykpcnrC1sFcQ5EGg
14       SQ0j7bgSTazkVQlF5AnqyQ
15       wqu7ILomIOPSduRwoWp4AQ
16       P1fJb2WQ1mXoiudj8UE44w
17       8QlnAcjGE6dgfKTVoxrPvQ
18       t_gan0EXAw8csKIeFyazJw
19       3gmBc0qN_LtGbZAjTtHWZg
20       yHTLTr-QN-loO_7XoxS8qg
21       iQLLIdmJSt6LQHSjpZERLA
22       6ilJq_05xRgek_8qUp36-g
23       MKyk4F4HSzHF8v-4cYe3Ww
24       gClSqcwuLsgyZRCzhJc7Dg
25       1qCuOcks5HRv67OHovAVpg
26       sbW8qHJgzEIH42B0S-3New
27       W7ysp8GLCBrs0GXIVwAcBQ
28       McikHxxEqZ2X0joaRNKlaw
29       tv8cS4aaA1VDaInYgggb6g
                  ...          
55517   